# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [29]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform


#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [30]:
data = pd.read_csv('../data/customer-product-sales.csv')

In [31]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [32]:
total_quantity = data.groupby(["CustomerID", "ProductName"])["Quantity"].sum().reset_index()
total_quantity.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [33]:
product_customer_matrix = pd.pivot_table(total_quantity, columns="CustomerID", index="ProductName", values="Quantity").fillna(0)
product_customer_matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [34]:
customer_similarity = squareform(pdist(product_customer_matrix.T, "euclidean"))

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [35]:
customer_similarity_df = pd.DataFrame(1 / (1 + customer_similarity), index=product_customer_matrix.columns, columns=product_customer_matrix.columns)
top_5_customers = list(customer_similarity_df[33].sort_values(ascending=False)[1:].head().index)
print(top_5_customers)

[264, 3535, 3317, 2503, 3305]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [36]:
top_5_customer_records = total_quantity[total_quantity["CustomerID"].isin(top_5_customers)].reset_index()
top_5_customer_records.head()

,index,CustomerID,ProductName,Quantity
0,131,264,Apricots - Halves,1
1,132,264,Apricots Fresh,1
2,133,264,Bacardi Breezer - Tropical,1
3,134,264,Bagel - Plain,1
4,135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [37]:
top_5_customer_item_quantity = top_5_customer_records.groupby("ProductName").agg({"Quantity":"sum"}).reset_index().sort_values("Quantity",ascending=False)
top_5_customer_item_quantity.head()

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
186,Towels - Paper / Kraft,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [38]:
recommendation = pd.merge(top_5_customer_item_quantity, product_customer_matrix[33], on="ProductName").reset_index()
recommendation.columns = ["ProductName", "Quantity", "Purchased"] 

not_bought = recommendation[recommendation["Purchased"] == 0]

not_bought.head()

ValueError: Length mismatch: Expected axis has 4 elements, new values have 3 elements

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [16]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

NameError: name 'customer_products' is not defined

In [27]:
recommendations = {}
customersID = list(data["CustomerID"].unique())

In [28]:
for customer_id in customersID:
  
    customer_similarity_df = pd.DataFrame(1 / (1 + customer_similarity), index=product_customer_matrix.columns, columns=product_customer_matrix.columns)
    top_5_customers = list(customer_similarity_df[customer_id].sort_values(ascending=False)[1:].head().index)
   
    top_5_customer_records = total_quantity[total_quantity["CustomerID"].isin(top_5_customers)].reset_index()

    top_5_customer_item_quantity = top_5_customer_records.groupby("ProductName").agg({"Quantity":"sum"}).reset_index().sort_values("Quantity",ascending=False)

    recommendation = pd.merge(top_5_customer_item_quantity, product_customer_matrix[customer_id], on="ProductName").reset_index()
    recommendation.columns = ["ProductName", "Quantity", "Purchased"] 
    not_bought = recommendation[recommendation["Purchased"] == 0].head(5)
    
    recommendations.update({customer: [not_bought["ProductName"]] for customer in set(data["CustomerID"])})

ValueError: Length mismatch: Expected axis has 4 elements, new values have 3 elements

In [24]:
recommendations

{83973: [0            Pepper - Black, Whole
  1            Knife Plastic - White
  2                Butter - Unsalted
  3    Wine - Ej Gallo Sierra Valley
  4    Wine - Blue Nun Qualitatswein
  Name: ProductName, dtype: object],
 59399: [0            Pepper - Black, Whole
  1            Knife Plastic - White
  2                Butter - Unsalted
  3    Wine - Ej Gallo Sierra Valley
  4    Wine - Blue Nun Qualitatswein
  Name: ProductName, dtype: object],
 92168: [0            Pepper - Black, Whole
  1            Knife Plastic - White
  2                Butter - Unsalted
  3    Wine - Ej Gallo Sierra Valley
  4    Wine - Blue Nun Qualitatswein
  Name: ProductName, dtype: object],
 49159: [0            Pepper - Black, Whole
  1            Knife Plastic - White
  2                Butter - Unsalted
  3    Wine - Ej Gallo Sierra Valley
  4    Wine - Blue Nun Qualitatswein
  Name: ProductName, dtype: object],
 18441: [0            Pepper - Black, Whole
  1            Knife Plastic - White
  2

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [15]:
recommendationsDF = pd.DataFrame(recommendations).reset_index().T
recommendationsDF.head()

,0
index,0
83973,"0 Butter - Unsalted 1 Wine - Ej Gallo Sierra Valley 3 Soup - Campbells Bean Medley 4 Wine - Blue Nun Qualitatswein 6 Chicken - Soup Base Name: ProductName, dtype: object"
59399,"0 Butter - Unsalted 1 Wine - Ej Gallo Sierra Valley 3 Soup - Campbells Bean Medley 4 Wine - Blue Nun Qualitatswein 6 Chicken - Soup Base Name: ProductName, dtype: object"
92168,"0 Butter - Unsalted 1 Wine - Ej Gallo Sierra Valley 3 Soup - Campbells Bean Medley 4 Wine - Blue Nun Qualitatswein 6 Chicken - Soup Base Name: ProductName, dtype: object"
49159,"0 Butter - Unsalted 1 Wine - Ej Gallo Sierra Valley 3 Soup - Campbells Bean Medley 4 Wine - Blue Nun Qualitatswein 6 Chicken - Soup Base Name: ProductName, dtype: object"


In [ ]:
recommendationsDF = pd.DataFrame.from_dict(recommendations, orient="index").reset_index()

recommendationsDF.columns = ['CustomerID', "Product 1", "Product 2","Product 3","Product 4","Product 5"]
recommendationsDF.head()

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.